In [1]:
import datetime
import json
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm as tqdm_notebook

In [2]:
def analyse(full_history_data_frame : pd.DataFrame, symbol : str) -> bool:
    today : str = str(datetime.datetime.now().date())
    one_year_ago = str((datetime.datetime.now() - datetime.timedelta(days=365)).date())
    history_data_frame = full_history_data_frame[full_history_data_frame['symbol'] == symbol]
    history_data_frame['date'] = pd.to_datetime(history_data_frame['date'])
    one_year_data_frame = history_data_frame.loc[(history_data_frame['date'] >= one_year_ago) & (history_data_frame['date'] <= today) & (history_data_frame['volume'] > 0)]
    print(symbol, one_year_ago, today, len(one_year_data_frame))
    if len(one_year_data_frame) < 200:
        return False
    # Past
    history_data_frame["price_three_days_ago"] = history_data_frame["close"].shift(3)
    history_data_frame["price_three_days_ago_difference"] = round(history_data_frame["close"] - history_data_frame["price_three_days_ago"], 2)
    history_data_frame["price_three_days_ago_return"] = round(history_data_frame["price_three_days_ago_difference"] / history_data_frame["price_three_days_ago"], 2)
    history_data_frame["price_two_days_ago"] = history_data_frame["close"].shift(2)
    history_data_frame["price_yesterday"] = history_data_frame["close"].shift(1)
    # Future
    history_data_frame["price_tomorrow"] = history_data_frame["close"].shift(-1)
    history_data_frame["price_tomorrow_difference"] = round(history_data_frame["price_tomorrow"] - history_data_frame["close"], 2)
    history_data_frame["price_tomorrow_return"] = round(history_data_frame["price_tomorrow_difference"] / history_data_frame["close"], 2)
    # Direction
    history_data_frame["direction"] = [1 if history_data_frame.loc[ei, "price_tomorrow_difference"] > 0 else -1 for ei in history_data_frame.index]
    # Moving Average
    history_data_frame["MA5"] = round(history_data_frame["close"].rolling(5).mean(), 2)
    history_data_frame["MA10"] = round(history_data_frame["close"].rolling(10).mean(), 2)
    history_data_frame["MA20"] = round(history_data_frame["close"].rolling(20).mean(), 2)
    history_data_frame["MA50"] = round(history_data_frame["close"].rolling(50).mean(), 2)
    history_data_frame["MA100"] = round(history_data_frame["close"].rolling(100).mean(), 2)
    history_data_frame["MA200"] = round(history_data_frame["close"].rolling(200).mean(), 2)
    # Indicator
    history_data_frame["MA50-1"] = history_data_frame["MA50"].shift(1)
    history_data_frame["MA200-1"] = history_data_frame["MA200"].shift(1)
    history_data_frame["golden-cross-1"] = history_data_frame["MA50-1"] <= history_data_frame["MA200-1"]
    history_data_frame["golden-cross-0"] = history_data_frame["MA50"] >= history_data_frame["MA200"]
    history_data_frame["golden-cross"] = history_data_frame["golden-cross-1"] & history_data_frame["golden-cross-0"]
    # Save to File
    plt.plot(history_data_frame[['close', 'MA50', 'MA200']].tail(200))
    plt.legend(['close', 'MA50', 'MA200'], loc='upper left')
    first : str = symbol[0].lower()
    plt.savefig(f"./images/{first}/{symbol}.png")
    plt.clf()
    if True in list(history_data_frame["golden-cross"].tail(5)):
        print(f"There is golden cross in {symbol}")
    return True in list(history_data_frame["golden-cross"].tail(5))

In [3]:
full_history_data_frame = pd.read_csv("./data/history.csv")
full_history_data_frame

,symbol,date,timestamp,open,high,low,close,volume
0,RDP,2009-09-22,1253577600,4.730,4.730,4.730,4.730,110
1,RDP,2009-09-23,1253664000,4.968,4.968,4.968,4.968,20
2,RDP,2009-09-24,1253750400,5.205,5.205,5.205,5.205,1220
3,RDP,2009-09-25,1253836800,5.440,5.440,5.440,5.440,440
4,RDP,2009-09-28,1254096000,5.704,5.704,5.704,5.704,7990
...,...,...,...,...,...,...,...,...
4021637,WTC,2023-11-28,1701129600,13.000,13.000,13.000,13.000,1100
4021638,WTC,2023-11-29,1701216000,13.000,13.000,13.000,13.000,1
4021639,WTC,2023-11-30,1701302400,13.000,13.000,13.000,13.000,0
4021640,WTC,2023-12-01,1701388800,13.500,13.500,13.500,13.500,100


In [4]:
vn30_file = open("./vn30.json", 'r')
vn30 : list[str] = json.load(vn30_file)

golden_cross_vn30 = []


for symbol in tqdm_notebook(vn30):
    golden_cross = analyse(full_history_data_frame, symbol)
    if golden_cross:
        golden_cross_vn30.append(symbol)

golden_cross_vn30

  0%|          | 0/30 [00:00<?, ?it/s]

/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_data_frame['date'] = pd.to_datetime(history_data_frame['date'])
/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_data_frame["price_three_days_ago"] = history_data_frame["close"].shift(3)
/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:12: SettingWithCopyWarning: 

ACB 2022-12-05 2023-12-05 251


/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_data_frame['date'] = pd.to_datetime(history_data_frame['date'])
/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_data_frame["price_three_days_ago"] = history_data_frame["close"].shift(3)
/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:12: SettingWithCopyWarning: 

BCM 2022-12-05 2023-12-05 251


...

/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_data_frame['date'] = pd.to_datetime(history_data_frame['date'])
/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_data_frame["price_three_days_ago"] = history_data_frame["close"].shift(3)
/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:12: SettingWithCopyWarning: 

VPB 2022-12-05 2023-12-05 251
VRE 2022-12-05 2023-12-05 251


/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_data_frame['date'] = pd.to_datetime(history_data_frame['date'])
/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_data_frame["price_three_days_ago"] = history_data_frame["close"].shift(3)
/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:12: SettingWithCopyWarning: 

[]

<Figure size 640x480 with 0 Axes>

In [5]:
companies_data_frame = pd.read_csv("./data/companies.csv")
vnindex : list[str] = list(companies_data_frame['stock_code'])
vnindex = list(filter(lambda s: len(s) == 3, vnindex))
vnindex

['A32',
 'AAA',
 'AAM',
 'AAS',
 'AAT',
 'AAV',
 'ABB',
 'ABC',
 'ABI',
 'ABR',
 'ABS',
 'ABT',
 'ABW',
 'ACB',
 'ACC',
 'ACE',
 'ACG',
 'ACL',
 'ACM',
 'ACS',
 'ACV',
 'ADC',
 'ADG',
 'ADP',
 'ADS',
 'AFX',
 'AG1',
 'AGE',
 'AGF',
 'AGG',
 'AGM',
 'AGP',
 'AGR',
 'AGX',
 'AIC',
 'ALT',
 'ALV',
 'AMC',
 'AMD',
 'AME',
 'AMP',
 'AMS',
 'AMV',
 'ANT',
 'ANV',
 'APC',
 'APF',
 'APG',
 'APH',
 'API',
 'APL',
 'APP',
 'APS',
 'APT',
 'ARM',
 'ART',
 'ASA',
 'ASG',
 'ASM',
 'ASP',
 'AST',
 'ATA',
 'ATB',
 'ATG',
 'ATS',
 'AUM',
 'AVC',
 'AVF',
 'B82',
 'BAB',
 'BAF',
 'BAL',
 'BAX',
 'BBC',
 'BBH',
 'BBM',
 'BBS',
 'BBT',
 'BCA',
 'BCB',
 'BCC',
 'BCE',
 'BCF',
 'BCG',
 'BCM',
 'BCO',
 'BCP',
 'BCV',
 'BDB',
 'BDG',
 'BDT',
 'BDW',
 'BED',
 'BEL',
 'BFC',
 'BGW',
 'BHA',
 'BHC',
 'BHG',
 'BHI',
 'BHK',
 'BHN',
 'BHP',
 'BHT',
 'BIC',
 'BID',
 'BIG',
 'BII',
 'BIO',
 'BKC',
 'BKG',
 'BKH',
 'BLF',
 'BLI',
 'BLN',
 'BLT',
 'BLW',
 'BMC',
 'BMD',
 'BMF',
 'BMG',
 'BMI',
 'BMJ',
 'BMN',
 'BMP',


In [6]:
golden_cross_vnindex = []

for symbol in tqdm_notebook(vnindex):
    golden_cross = analyse(full_history_data_frame, symbol)
    if golden_cross:
        golden_cross_vnindex.append(symbol)

golden_cross_vnindex

  0%|          | 0/1617 [00:00<?, ?it/s]

/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_data_frame['date'] = pd.to_datetime(history_data_frame['date'])


A32 2022-12-05 2023-12-05 99
AAA 2022-12-05 2023-12-05 251


/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_data_frame['date'] = pd.to_datetime(history_data_frame['date'])
/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_data_frame["price_three_days_ago"] = history_data_frame["close"].shift(3)
/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:12: SettingWithCopyWarning: 

AAM 2022-12-05 2023-12-05 249


...

/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_data_frame['date'] = pd.to_datetime(history_data_frame['date'])
/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_data_frame['date'] = pd.to_datetime(history_data_frame['date'])
/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:5: SettingWithCopyWarning: 
A value is 

HLB 2022-12-05 2023-12-05 80
HLC 2022-12-05 2023-12-05 246


/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_data_frame['date'] = pd.to_datetime(history_data_frame['date'])
/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_data_frame["price_three_days_ago"] = history_data_frame["close"].shift(3)
/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:12: SettingWithCopyWarning: 

HLD 2022-12-05 2023-12-05 251


/var/folders/jh/z981c7zj0vz0gmyfc8mhdxdr0000gn/T/ipykernel_14461/12777537.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_data_frame['date'] = pd.to_datetime(history_data_frame['date'])


In [ ]:
for symbol in golden_cross_vnindex:
    first : str = symbol[0].lower()
    print(f"./images/{first}/{symbol}.png")